In [4]:
import requests
import os
import torch
import pandas as pd
import utils_clusters
import torch
from tqdm import tqdm
import importlib

In [ ]:
## Change the paths below to point to the relevant directory

In [ ]:
PATH_TO_POOLED_BERT_EMBEDDINGS = '../protBERT/post_pooling_seq_vectors/pool_parti/'
PATH_TO_POOLED_ESM_EMBEDDINGS = '/oak/stanford/groups/rbaltman/esm_embeddings/esm2_t33_650M_uniprot/post_pooling_seq_vectors/PR_max_of_max_attn_no_prune/'

In [ ]:
# downloading the data from Berkeley SCOPe database
! wget https://zenodo.org/records/5829561/files/dir.cla.scope.2.08-stable.txt?download=1

In [9]:
df_scop = pd.read_csv("dir.cla.scope.2.08-stable.txt", sep='\t', skiprows=[0,1,2,3], header=None)

In [10]:
df_scop

,0,1,2,3,4,5
0,d1ux8a_,1ux8,A:,a.1.1.1,113449,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
1,d1dlwa_,1dlw,A:,a.1.1.1,14982,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
2,d1uvya_,1uvy,A:,a.1.1.1,100068,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
3,d1dlya_,1dly,A:,a.1.1.1,14983,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
4,d1uvxa_,1uvx,A:,a.1.1.1,100067,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
...,...,...,...,...,...,...
344846,d2ehoc3,2eho,C:0-0,l.1.1.1,286555,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."
344847,d2ehof3,2eho,F:0-0,l.1.1.1,286556,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."
344848,d2ehog3,2eho,G:0-0,l.1.1.1,286557,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."
344849,d2ehok3,2eho,K:0-0,l.1.1.1,286558,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."


In [13]:
def get_pdb_from_uniprot(uniprot_id):
    """
    Retrieve the PDB ID for a given UniProt ID using the UniProt API.
    :param uniprot_id: The UniProt ID of the protein.
    :return: A list of PDB IDs corresponding to the UniProt ID.
    """
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.xml"
    response = requests.get(url)
    
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data for UniProt ID {uniprot_id}")
    
    # Parse the XML to find the PDB entries
    pdb_ids = []
    if "<dbReference type=\"PDB\"" in response.text:
        # Extract all PDB IDs from the XML response
        lines = response.text.splitlines()
        for line in lines:
            if 'dbReference type="PDB"' in line:
                pdb_id_start = line.find('id="') + 4
                pdb_id_end = line.find('"', pdb_id_start)
                pdb_id = line[pdb_id_start:pdb_id_end]
                pdb_ids.append(pdb_id)
    return pdb_ids

def get_scop_labels_from_scop_df(pdb_id):
    """
    Retrieve SCOPe classification labels for a given PDB ID using the SCOPe API.
    :param pdb_id: The PDB ID of the protein structure.
    :return: SCOPe classification labels.
    """
    pdb_id = pdb_id.lower()
    if pdb_id.lower() in df_scop[1].values:
        return df_scop[df_scop[1] == pdb_id][3].values[0]
    else:
        return None

def get_scop_labels_for_uniprot(uniprot_id):
    """
    Get the SCOP/SCOPe labels for a protein with a given UniProt ID.
    :param uniprot_id: The UniProt ID of the protein.
    :return: SCOP/SCOPe classification labels for the protein.
    """
    # Step 1: Get the PDB IDs for the UniProt ID
    pdb_ids = get_pdb_from_uniprot(uniprot_id)
    
    if not pdb_ids:
        #raise Exception(f"No PDB IDs found for UniProt ID {uniprot_id}")
        return None
    
    # Step 2: For each PDB ID, get the SCOPe labels
    #all_scop_labels = {}
    all_scop_labels = []
    for pdb_id in pdb_ids:
        try:
            scop_labels = get_scop_labels_from_scop_df(pdb_id)
            if scop_labels is not None:
                #all_scop_labels[pdb_id] = scop_labels
                all_scop_labels.append(scop_labels)
        except Exception as e:
            #print(f"Error fetching SCOPe for {pdb_id}: {e}")
            #all_scop_labels[pdb_id] = None
            pass
    
    return list(set(all_scop_labels))


In [11]:
all_embedded_bert = os.listdir("/oak/stanford/groups/rbaltman/protbert_embeddings/post_pooling_seq_vectors/PR_max_of_max_attn_no_prune/")
all_embedded_bert = [file[:-3] for file in all_embedded_bert]

In [14]:
all_embedded_esm = os.listdir("/oak/stanford/groups/rbaltman/esm_embeddings/esm2_t33_650M_uniprot/post_pooling_seq_vectors/PR_max_of_max_attn_no_prune/")
all_embedded_esm = [file[:-3] for file in all_embedded_esm]

In [ ]:
df_acc_to_scop = {}
for acc in tqdm(all_embedded_bert):
    scop_labels = get_scop_labels_for_uniprot(acc)
    if scop_labels is None:
        continue
    df_acc_to_scop[acc] = scop_labels

 23%|██▎       | 4716/20777 [2:03:08<6:32:58,  1.47s/it] 

In [15]:
import multiprocessing
from tqdm import tqdm

# Define the worker function
def process_acc(acc):
    try:
        # Check if the acc is already in the df_acc_to_scop dictionary
        if acc in df_acc_to_scop:
            return (acc, df_acc_to_scop[acc])

        # Fetch scop labels
        scop_labels = get_scop_labels_for_uniprot(acc)
        if scop_labels is None:
            return (acc, None)
        
        return (acc, scop_labels)

    except Exception as e:
        # Log the error or print it (optional)
        print(f"Error processing {acc}: {e}")
        # Return None for this acc to avoid crashing the entire process
        return (acc, None)

# Function to initialize the dictionary using multiprocessing
def create_dict_with_multiprocessing(all_embedded_esm):
    # Number of available CPUs
    num_cpus = multiprocessing.cpu_count()
    print(f"using {num_cpus} cpus")

    # Creating a pool of workers
    with multiprocessing.Pool(num_cpus) as pool:
        # Map the function over all elements and create an iterator to collect results
        results = pool.imap(process_acc, all_embedded_esm)
        
        # Initialize progress bar
        results_with_progress = tqdm(results, total=len(all_embedded_esm))
        
        # Collect results into a dictionary
        df_acc_to_scop_esm = {acc: scop_labels for acc, scop_labels in results_with_progress if scop_labels is not None}
        
    return df_acc_to_scop_esm

In [16]:
# Usage
df_acc_to_scop = torch.load("df_acc_to_scop.pt")
df_acc_to_scop_esm = create_dict_with_multiprocessing(all_embedded_esm)

using 24 cpus


  6%|▌         | 1194/21083 [00:58<23:23, 14.17it/s] 

Error processing P22455-3: Failed to fetch data for UniProt ID P22455-3


100%|██████████| 21083/21083 [18:21<00:00, 19.14it/s]   


In [16]:
torch.save(df_acc_to_scop, "df_acc_to_scop.pt")

In [1]:
len(df_acc_to_scop)

NameError: name 'df_acc_to_scop' is not defined

In [17]:
df_acc_to_scop_1to1 = {}
for acc in df_acc_to_scop:
    if len(df_acc_to_scop[acc]) == 1:
        df_acc_to_scop_1to1[acc] = df_acc_to_scop[acc][0]

In [17]:
df_acc_to_scop_1to1_esm = {}
for acc in df_acc_to_scop_esm:
    if len(df_acc_to_scop_esm[acc]) == 1:
        df_acc_to_scop_1to1_esm[acc] = df_acc_to_scop_esm[acc][0]

In [19]:
len(df_acc_to_scop_1to1)

1893

In [144]:
len(df_acc_to_scop_1to1_esm)

2963

In [20]:
torch.save(df_acc_to_scop_1to1, "df_acc_to_scop_1to1.pt")

In [18]:
torch.save(df_acc_to_scop_1to1_esm, "df_acc_to_scop_1to1_esm.pt")

In [19]:
importlib.reload(utils_clusters)

<module 'utils_clusters' from '/oak/stanford/groups/rbaltman/alptartici/pooling_work/pooling_performances/dimensionality_reduction_analysis/utils_clusters.py'>

In [11]:
df_acc_to_scop_1to1 = torch.load("df_acc_to_scop_1to1.pt")

In [12]:
df_acc_to_scop_1to1_esm = torch.load("df_acc_to_scop_1to1_esm.pt")

In [13]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(df_acc_to_scop_1to1.items()), columns=['uniprot_acc', 'level_4'])

# Split 'level_4' into 'level_1', 'level_2', 'level_3'
df['level_1'] = df['level_4'].apply(lambda x: x.split('.')[0])  # First part (e.g., 'c')
df['level_2'] = df['level_4'].apply(lambda x: '.'.join(x.split('.')[:2]))  # First two parts (e.g., 'c.55')
df['level_3'] = df['level_4'].apply(lambda x: '.'.join(x.split('.')[:3]))  # First three parts (e.g., 'c.55.1')

In [14]:
# Convert the dictionary to a DataFrame
df_esm = pd.DataFrame(list(df_acc_to_scop_1to1_esm.items()), columns=['uniprot_acc', 'level_4'])

# Split 'level_4' into 'level_1', 'level_2', 'level_3'
df_esm['level_1'] = df_esm['level_4'].apply(lambda x: x.split('.')[0])  # First part (e.g., 'c')
df_esm['level_2'] = df_esm['level_4'].apply(lambda x: '.'.join(x.split('.')[:2]))  # First two parts (e.g., 'c.55')
df_esm['level_3'] = df_esm['level_4'].apply(lambda x: '.'.join(x.split('.')[:3]))  # First three parts (e.g., 'c.55.1')

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
PLMs = ['protbert', 'esm']
levels = ['level_1', 'level_2']
metrics = ["euclidean", "cosine"]
pooling_methods = ["PR_max_of_max_attn_no_prune", "mean_pooled", "max_pooled", "cls_pooled"]

In [100]:
filename = "scop_distance_ratios_with_abfgj_in_level1.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    ratio = utils_clusters.evaluate_embeddings(df_filt_for_level1, 
                                                               'uniprot_acc', 
                                                               level, PLM, 
                                                               pooling, 
                                                               distance_metric=metric)['ratio_of_inter_to_intra']
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{ratio}\n")

In [101]:
level_1_to_keep_abcdfgj = ['a', 'b', 'c','d','f', 'g', 'j']
df_filt_for_level1_abcdfgj = df[df['level_1'].isin(level_1_to_keep_abcdfgj)]
df_filt_for_level1_abcdfgj = df_filt_for_level1_abcdfgj.reset_index()

In [102]:
filename = "scop_distance_ratios_with_abcdfgj_in_level1.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    ratio = utils_clusters.evaluate_embeddings(df_filt_for_level1_abcdfgj, 
                                                               'uniprot_acc', 
                                                               level, PLM, 
                                                               pooling, 
                                                               distance_metric=metric)['ratio_of_inter_to_intra']
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{ratio}\n")

In [103]:
level_1_to_keep_abcdefgj = ['a', 'b', 'c','d', "e", 'f', 'g', 'j']
df_filt_for_level1_abcdefgj = df[df['level_1'].isin(level_1_to_keep_abcdefgj)]
df_filt_for_level1_abcdefgj = df_filt_for_level1_abcdefgj.reset_index()

In [105]:
filename = "scop_distance_ratios_with_abcdefgj_in_level1.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    ratio = utils_clusters.evaluate_embeddings(df_filt_for_level1_abcdefgj, 
                                                               'uniprot_acc', 
                                                               level, PLM, 
                                                               pooling, 
                                                               distance_metric=metric)['ratio_of_inter_to_intra']
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{ratio}\n")

In [107]:
df_filt_for_level1_abcdefgj['level_1'].value_counts()

level_1
b    468
a    466
d    399
c    355
g    129
j     29
e     20
f     16
Name: count, dtype: int64

In [108]:
df_filt_for_level1_abcdefgj['level_2'].value_counts()

level_2
b.1      127
c.37      89
d.144     67
a.4       65
b.34      47
        ... 
d.209      1
a.194      1
b.95       1
c.110      1
c.84       1
Name: count, Length: 305, dtype: int64

In [110]:
len(df_filt_for_level1_abcdefgj)

1882

In [117]:
importlib.reload(utils_clusters)

<module 'utils_clusters' from '/oak/stanford/groups/rbaltman/alptartici/pooling_work/pooling_performances/dimensionality_reduction_analysis/utils_clusters.py'>

In [112]:
uniprot_col = 'uniprot_acc'
level = 'level_1'
PLM = 'protbert'
pooling = 'cls_pooled'
metric = "euclidean"

In [122]:
result = utils_clusters.evaluate_embeddings_for_dist_ratio(df_filt_for_level1_abcdefgj,
                                                  uniprot_col = uniprot_col,
                                                  label_col = level,
                                                  plm_type = PLM,
                                                  pooling_method = pooling,
                                                  distance_metric=metric,
                                                  verbose=True,
                                                  sampling_mode = True,
                                                  num_iterations = 2,
                                                  sample_size = 500,
                                                  min_samples_per_class=5
                                                 )

In [125]:
result["ratio_of_inter_to_intra_mean"]

1.8950032580632064

# sampled and looped

In [73]:
importlib.reload(utils_clusters)

<module 'utils_clusters' from '/oak/stanford/groups/rbaltman/alptartici/pooling_work/pooling_performances/dimensionality_reduction_analysis/utils_clusters.py'>

In [82]:
PLMs = ['protbert', 'esm']
levels = ['level_1', 'level_2']
metrics = ["euclidean", "cosine"]
pooling_methods = ["PR_max_of_max_attn_no_prune", "mean_pooled", "max_pooled", "cls_pooled", "sum_pooled"]
uniprot_col = 'uniprot_acc'

In [16]:
level_1_to_keep_abcdefgj = ['a', 'b', 'c','d', "e", 'f', 'g', 'j']
df_filt_for_level1_abcdefgj = df[df['level_1'].isin(level_1_to_keep_abcdefgj)]
df_filt_for_level1_abcdefgj = df_filt_for_level1_abcdefgj.reset_index()
df_filt_for_level1_abcdefgj_esm = df_esm[df_esm['level_1'].isin(level_1_to_keep_abcdefgj)]
df_filt_for_level1_abcdefgj_esm = df_filt_for_level1_abcdefgj_esm.reset_index()

In [90]:
num_iter = 20
filename = f"scop_subsampled_distance_ratios_with_abcdefgj_in_level1_{num_iter}iterations.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        if PLM == 'esm':
            df_to_use = df_filt_for_level1_abcdefgj_esm
        else:
            df_to_use = df_filt_for_level1_abcdefgj
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    result = utils_clusters.evaluate_embeddings_for_dist_ratio(df_to_use,
                                                  uniprot_col = uniprot_col,
                                                  label_col = level,
                                                  plm_type = PLM,
                                                  pooling_method = pooling,
                                                  distance_metric=metric,
                                                  verbose=True,
                                                  sampling_mode = True,
                                                  num_iterations = num_iter,
                                                  sample_size = int(len(df_to_use)//num_iter*2),
                                                  min_samples_per_class=5
                                                 )
                    mean = result["ratio_of_inter_to_intra_mean"]
                    std = result["ratio_of_inter_to_intra_std"]
                    ratios = result["ratios"]
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{mean} +/- {std} &{ratios}\n")

Embedding is None for Q92889
Embedding is None for P22415
Embedding is None for Q9BUI4
Embedding is None for Q9UKC9
Embedding is None for P68363
Embedding is None for Q9Y6Y1
Embedding is None for P13489
Embedding is None for P06132
Embedding is None for Q7L273
Embedding is None for P50440
Embedding is None for Q9HBH9
Embedding is None for P23193


Processing UniProt Accessions:   0%|          | 0/297 [00:00<?, ?it/s]

Embedding is None for Q9Y2J2
Embedding is None for P51531
Embedding is None for Q9NRD5


Embedding is None for Q96GD3
Embedding is None for P36941
Embedding is None for P62280
Embedding is None for P13489
Embedding is None for O15519
Embedding is None for P36639
Embedding is None for P52888
Embedding is None for Q9NWB1
Embedding is None for Q8N5K1


Processing UniProt Accessions:   0%|          | 0/297 [00:00<?, ?it/s]

...

Processing UniProt Accessions:  23%|██▎       | 255/1090 [00:00<00:00, 2548.99it/s]

Embedding is None for P02008
Embedding is None for Q9UKC9
Embedding is None for O75390
Embedding is None for Q9Y2J2
Embedding is None for P30566
Embedding is None for P82979
Embedding is None for Q9H074
Embedding is None for Q96EY1
Embedding is None for Q8IX01
Embedding is None for P18858
Embedding is None for P52657
Embedding is None for P61244
Embedding is None for P22415
Embedding is None for P55347
Embedding is None for P53814
Embedding is None for P43897
Embedding is None for P12429
Embedding is None for Q9UI12
Embedding is None for P51946
Embedding is None for P12277
Embedding is None for P68363
Embedding is None for P40394
Embedding is None for Q8TBB1
Embedding is None for Q9NSC5
Embedding is None for P13489
Embedding is None for O15519
Embedding is None for P55212


Processing UniProt Accessions:  93%|█████████▎| 1009/1090 [00:00<00:00, 2483.74it/s]

Embedding is None for Q71U36
Embedding is None for Q8WUI4
Embedding is None for O95671
Embedding is None for Q16831
Embedding is None for P50213
Embedding is None for O95630
Embedding is None for Q9Y6U3
Embedding is None for P50440
Embedding is None for P49773
Embedding is None for Q9Y237
Embedding is None for O43617
Embedding is None for O75569
Embedding is None for P18615
Embedding is None for P09543
Embedding is None for Q9Y2K5
Embedding is None for Q6Y7W6
Embedding is None for O75173
Embedding is None for P52888
Embedding is None for P16591
Embedding is None for Q92934
Embedding is None for Q07654
Embedding is None for O60711
Embedding is None for P23193
Embedding is None for Q9NXR8


Embedding is None for Q8N5K1


# statistical analysis

In [20]:
import re
from collections import defaultdict
import scipy.stats as stats

# Initialize a dictionary to hold the values for each PLM, level, metric, and pooling type
data_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

# Function to process the file content
def process_file(file_name):
    with open(file_name, 'r') as f:
        for line in f:
            # Split the line based on the tab separator
            parts = line.strip().split('\t')
            if len(parts) == 5:
                plm, level, metric, pooling, mean_std_ratios = parts
                
                # Extract mean, std and ratios using regular expression
                match = re.match(r"(.+?) \+/- (.+?) &(.+)", mean_std_ratios)
                if match:
                    mean, std, ratios = match.groups()
                    
                    # Store the values in the dictionary
                    data_dict[plm][level][metric][pooling] = {
                        'mean': float(mean),
                        'std': float(std),
                        'ratios': ratios
                    }
    return data_dict

def run_significance_test(performance_data1, performance_data2, test_type='mannwhitney', alternative='greater'):

    performance_data1 = performance_data1[1:-1].split(', ')
    performance_data1 = [float(data) for data in performance_data1]

    performance_data2 = performance_data2[1:-1].split(', ')
    performance_data2 = [float(data) for data in performance_data2]
    
    if test_type == 'wilcoxon':
        # Perform Wilcoxon signed-rank test
        statistic, p_value = stats.wilcoxon(performance_data1, performance_data2)
    elif test_type == 'mannwhitney':
        # Perform Mann-Whitney U test
        statistic, p_value = stats.mannwhitneyu(performance_data1, performance_data2, alternative=alternative)
    else:
        raise ValueError("Invalid test_type. Choose either 'wilcoxon' or 'mannwhitney'.")
    
    return {
        'statistic': statistic,
        'p_value': p_value,
        'significant': p_value < 0.05,
        'test_type': test_type
    }

In [21]:
# Example file name provided by user
file_name = "scop_subsampled_distance_ratios_with_abcdefgj_in_level1_20iterations.txt"  # Replace with the actual file path
file_name = f"output/{file_name}"
# Process the file
data_abcdefgj = process_file(file_name)

## abcdefgj

### ProtBERT

#### level 2

In [24]:
data = data_abcdefgj
PLM = 'protbert'
level = 'level_2'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [25]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 267.0,
 'p_value': 0.03602271523367195,
 'significant': True,
 'test_type': 'mannwhitney'}

In [26]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 400.0,
 'p_value': 3.397807564086679e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [27]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 400.0,
 'p_value': 3.397807564086679e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [28]:
method_2 = 'sum_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 392.0,
 'p_value': 1.1088821650461217e-07,
 'significant': True,
 'test_type': 'mannwhitney'}

### ESM

#### level 2

In [22]:
data = data_abcdefgj
PLM = 'esm'
level = 'level_2'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [146]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 394.0,
 'p_value': 8.285396466929393e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [147]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 398.0,
 'p_value': 4.586386355828241e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [148]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 400.0,
 'p_value': 3.397807564086679e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [23]:
method_2 = 'sum_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 369.0,
 'p_value': 2.5828894122485707e-06,
 'significant': True,
 'test_type': 'mannwhitney'}

In [151]:
data = data_abfgj
PLM = 'protbert'
level = 'level_1'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [152]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 263.0,
 'p_value': 0.045453691625313236,
 'significant': True,
 'test_type': 'mannwhitney'}

In [153]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 310.0,
 'p_value': 0.0015283146140873955,
 'significant': True,
 'test_type': 'mannwhitney'}

In [154]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 399.0,
 'p_value': 3.9490168333373314e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [166]:
data = data_abfgj
PLM = 'esm'
level = 'level_1'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [167]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 259.0,
 'p_value': 0.056775696491720214,
 'significant': False,
 'test_type': 'mannwhitney'}

In [168]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 144.0,
 'p_value': 0.9367846913166159,
 'significant': False,
 'test_type': 'mannwhitney'}

In [169]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 397.0,
 'p_value': 5.322844918751244e-08,
 'significant': True,
 'test_type': 'mannwhitney'}